In [7]:
import os
import polars as pl

# Import Dataset

In [8]:
local_path='dataset/soccer18-19.csv'
auto='auto'

In [9]:
# get current directory
path = os.getcwd()
print("Current Directory", path)
print()
 
# parent directory
parent = os.path.dirname(path)
print("Parent directory", parent)
print()

# dataset path
dataset_path=os.path.join(parent,local_path)
print("Dataset path",dataset_path)
print()

# auto path
path_auto=os.path.join(parent,auto)
print("Dump path",path_auto)

Current Directory /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/football/notebook

Parent directory /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/football

Dataset path /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/football/dataset/soccer18-19.csv

Dump path /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/football/auto


In [10]:
df=pl.read_csv(dataset_path)

print("shape:", df.shape)

df.head(5)

shape: (380, 23)


Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
str,str,str,str,i64,i64,str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""E0""","""10/8/18""","""Man United""","""Leicester""",2,1,"""H""",1,0,"""H""","""A Marriner""",8,13,6,4,11,8,2,5,2,1,0,0
"""E0""","""11/8/18""","""Bournemouth""","""Cardiff""",2,0,"""H""",1,0,"""H""","""K Friend""",12,10,4,1,11,9,7,4,1,1,0,0
"""E0""","""11/8/18""","""Fulham""","""Crystal Palace…",0,2,"""A""",0,1,"""A""","""M Dean""",15,10,6,9,9,11,5,5,1,2,0,0
"""E0""","""11/8/18""","""Huddersfield""","""Chelsea""",0,3,"""A""",0,2,"""A""","""C Kavanagh""",6,13,1,4,9,8,2,5,2,1,0,0
"""E0""","""11/8/18""","""Newcastle""","""Tottenham""",1,2,"""A""",1,2,"""A""","""M Atkinson""",15,15,2,5,11,12,3,5,2,2,0,0


# Challenges

 ## Maximum number of fouls in one game

In [11]:
#max_foul=df.select(pl.max("HF","AF")).select(pl.max("max"))[0]

max_foul=max(df['HF'].max(),df['AF'].max())

print('Maximum Fouls in one game:',max_foul)

Maximum Fouls in one game: 23


## Game in which maximum number of fouls happened

In [12]:
df.filter((df['HF'] == max_foul)|(df['AF'] == max_foul))

Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
str,str,str,str,i64,i64,str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""E0""","""20/10/2018""","""Wolves""","""Watford""",0,2,"""A""",0,2,"""A""","""L Mason""",10,9,1,3,23,13,8,2,3,1,0,0


## Team committing the highest number of fools in average

### Shape the data games per team

In [13]:

# Home team reshape
col_home=['Div','Date','Team','Oponent','Oponent_FTG','FTG','FTR','Oponent_HTG','HTG','HTR','Referee','Oponent_S','S','Oponent_ST','ST','Oponent_F','F','Oponent_C','C','Oponent_Y','Y','Oponent_R','R','Venue']
rename_col_home={
    'Div':'Div',
    'HomeTeam':'Team',
    'AwayTeam':'Oponent',
    'FTHG':'FTG',
    'HTHG':'HTG',
    'HS':'S',
    'HST':'ST',
    'HF':'F',
    'HC':'C',
    'HY':'Y',
    'HR':'R',
    'FTAG':'Oponent_FTG',
    'HTAG':'Oponent_HTG',
    'AS':'Oponent_S',
    'AST':'Oponent_ST',
    'AF':'Oponent_F',
    'AC':'Oponent_C',
    'AY':'Oponent_Y',
    'AR':'Oponent_R' 
    }

df_home=df.with_columns(
    pl.when(pl.col('HTR')=='H').then('WIN')
    .otherwise(
        (pl.when(pl.col('HTR')=='A').then('LOSE'))
            .otherwise('DRAW')
    )
    .alias('HTR'),
    pl.when(pl.col('FTR')=='H').then('WIN')
    .otherwise(
        (pl.when(pl.col('FTR')=='A').then('LOSE'))
            .otherwise('DRAW')
    )
    .alias('FTR'),
    pl.lit('HOME').alias('Venue')
    ).rename(rename_col_home).select(col_home)

# Away Team reshape
col_away=['Div','Date','Team','Oponent','Oponent_FTG','FTG','FTR','Oponent_HTG','HTG','HTR','Referee','Oponent_S','S','Oponent_ST','ST','Oponent_F','F','Oponent_C','C','Oponent_Y','Y','Oponent_R','R','Venue']

rename_col_away={
    'Div':'Div',
    'HomeTeam':'Oponent',
    'AwayTeam':'Team',
    'FTAG':'FTG',
    'HTAG':'HTG',
    'AS':'S',
    'AST':'ST',
    'AF':'F',
    'AC':'C',
    'AY':'Y',
    'AR':'R',
    'FTHG':'Oponent_FTG',
    'HTHG':'Oponent_HTG',
    'HS':'Oponent_S',
    'HST':'Oponent_ST',
    'HF':'Oponent_F',
    'HC':'Oponent_C',
    'HY':'Oponent_Y',
    'HR':'Oponent_R' 
    }

df_away=df.with_columns(
    
    pl.when(pl.col('HTR')=='H').then('LOSE')
    .otherwise(
        (pl.when(pl.col('HTR')=='A').then('WIN'))
            .otherwise('DRAW')
    )
    .alias('HTR'),
    pl.when(pl.col('FTR')=='H').then('LOSE')
    .otherwise(
        (pl.when(pl.col('FTR')=='A').then('WIN'))
            .otherwise('DRAW')
    )
    .alias('FTR'),
    pl.lit('HOME').alias('Venue')
    ).rename(rename_col_away).select(col_away)


# Aggregate
df2=pl.concat([df_home,df_away])


df2.write_csv(os.path.join(path_auto,'df2.csv'))

print ('Shape:',df2.shape)

df2.head(5)


Shape: (760, 24)


/var/folders/07/v1ngwnc136b1lhw5lqnq_fjw0000gn/T/ipykernel_14384/1868965930.py:26: DeprecationWarning: in a future version, string input will be parsed as a column name rather than a string literal. To silence this warning, pass the input as an expression instead: `pl.lit('WIN')`
  pl.when(pl.col('HTR')=='H').then('WIN')
/var/folders/07/v1ngwnc136b1lhw5lqnq_fjw0000gn/T/ipykernel_14384/1868965930.py:28: DeprecationWarning: in a future version, string input will be parsed as a column name rather than a string literal. To silence this warning, pass the input as an expression instead: `pl.lit('LOSE')`
  (pl.when(pl.col('HTR')=='A').then('LOSE'))
/var/folders/07/v1ngwnc136b1lhw5lqnq_fjw0000gn/T/ipykernel_14384/1868965930.py:29: DeprecationWarning: in a future version, string input will be parsed as a column name rather than a string literal. To silence this warning, pass the input as an expression instead: `pl.lit('DRAW')`
  .otherwise('DRAW')
/var/folders/07/v1ngwnc136b1lhw5lqnq_fjw0000gn/

Div,Date,Team,Oponent,Oponent_FTG,FTG,FTR,Oponent_HTG,HTG,HTR,Referee,Oponent_S,S,Oponent_ST,ST,Oponent_F,F,Oponent_C,C,Oponent_Y,Y,Oponent_R,R,Venue
str,str,str,str,i64,i64,str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""E0""","""10/8/18""","""Man United""","""Leicester""",1,2,"""WIN""",0,1,"""WIN""","""A Marriner""",13,8,4,6,8,11,5,2,1,2,0,0,"""HOME"""
"""E0""","""11/8/18""","""Bournemouth""","""Cardiff""",0,2,"""WIN""",0,1,"""WIN""","""K Friend""",10,12,1,4,9,11,4,7,1,1,0,0,"""HOME"""
"""E0""","""11/8/18""","""Fulham""","""Crystal Palace…",2,0,"""LOSE""",1,0,"""LOSE""","""M Dean""",10,15,9,6,11,9,5,5,2,1,0,0,"""HOME"""
"""E0""","""11/8/18""","""Huddersfield""","""Chelsea""",3,0,"""LOSE""",2,0,"""LOSE""","""C Kavanagh""",13,6,4,1,8,9,5,2,1,2,0,0,"""HOME"""
"""E0""","""11/8/18""","""Newcastle""","""Tottenham""",2,1,"""LOSE""",2,1,"""LOSE""","""M Atkinson""",15,15,5,2,12,11,5,3,2,2,0,0,"""HOME"""


### Group per team, concentrate on fouls

In [19]:
df3=df2.group_by('Team').agg(
    pl.col('F').mean().alias('F_mean'),
    pl.col('F').min().alias('F_min'),
    pl.col('F').max().alias('F_max'),
    pl.col('F').std().alias('F_std')
).sort('F_mean')

df3.write_csv(os.path.join(path_auto,'df3.csv'))

df3.head(5)

Team,F_mean,F_min,F_max,F_std
str,f64,i64,i64,f64
"""Liverpool""",8.289474,3,17,3.819636
"""Man City""",8.631579,2,14,3.10554
"""West Ham""",8.710526,2,14,2.577532
"""Bournemouth""",8.921053,3,14,2.644541
"""Chelsea""",8.921053,5,15,2.529625


## Percentage of Home win vs Away Win vs Draw

In [20]:
df4=df.group_by('FTR').agg(
    pl.col('Div').count().alias('Number')
)

df4.write_csv(os.path.join(path_auto,'df4.csv'))


df4

FTR,Number
str,u32
"""H""",181
"""D""",71
"""A""",128


## Impact of Red cards on the win probablity

In [21]:
df5=df2.group_by(['R','FTR']).agg(
    pl.col('Div').count().alias('Count'),
)

df6=df5.pivot(values="Count", index="R", columns="FTR", aggregate_function="sum").fill_null(0)

df6.write_csv(os.path.join(path_auto,'df6.csv'))

df6.head

<bound method DataFrame.head of shape: (3, 4)
┌─────┬──────┬──────┬─────┐
│ R   ┆ LOSE ┆ DRAW ┆ WIN │
│ --- ┆ ---  ┆ ---  ┆ --- │
│ i64 ┆ u32  ┆ u32  ┆ u32 │
╞═════╪══════╪══════╪═════╡
│ 1   ┆ 25   ┆ 9    ┆ 11  │
│ 0   ┆ 283  ┆ 133  ┆ 298 │
│ 2   ┆ 1    ┆ 0    ┆ 0   │
└─────┴──────┴──────┴─────┘>

## Impact of Half Time Score on the Full Time Score

In [22]:
df7=df2.group_by(['HTR','FTR']).agg(
    pl.col('Div').count().alias('Count'),
).pivot(values="Count", index="HTR", columns="FTR", aggregate_function="sum").fill_null(0)

df7

HTR,WIN,LOSE,DRAW
str,u32,u32,u32
"""WIN""",186,17,29
"""DRAW""",106,106,84
"""LOSE""",17,186,29


## Half Time Decision making Dataset

In [23]:
col_half=['Div','Team','Oponent','Oponent_HTG','HTG','HTR','Referee','Venue','FTR']

df_half=df2.select(col_half)

df_half.write_csv(os.path.join(path_auto,'df_half.csv'))

df_half.head(5)

Div,Team,Oponent,Oponent_HTG,HTG,HTR,Referee,Venue,FTR
str,str,str,i64,i64,str,str,str,str
"""E0""","""Man United""","""Leicester""",0,1,"""WIN""","""A Marriner""","""HOME""","""WIN"""
"""E0""","""Bournemouth""","""Cardiff""",0,1,"""WIN""","""K Friend""","""HOME""","""WIN"""
"""E0""","""Fulham""","""Crystal Palace…",1,0,"""LOSE""","""M Dean""","""HOME""","""LOSE"""
"""E0""","""Huddersfield""","""Chelsea""",2,0,"""LOSE""","""C Kavanagh""","""HOME""","""LOSE"""
"""E0""","""Newcastle""","""Tottenham""",2,1,"""LOSE""","""M Atkinson""","""HOME""","""LOSE"""
